In [65]:
#importing useful packages
import pandas as pd
from google.colab import files
import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark-palette')
import datetime
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder

In [66]:
#USE THIS CELL TO PULL DATA FROM GOOGLE DRIVE, OR USE THE NEXT CELL TO UPLOAD A LOCAL FILE
#IF YOU HAVE THE FILE IN DRIVE IT'S A LOT FASTER
#mount google colab
from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/My Drive/urban economics/d03_text_station_hour_2020_03.txt')
peakTraffic = pd.read_csv('/content/gdrive/My Drive/urban economics/peak_traffic.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
print('Shape of hourly-interval dataframe: {}'.format(df.shape))
print(df.columns)
print('Shape of peak traffic dataframe: {}'.format(peakTraffic.shape))
print(peakTraffic.columns)


Shape of hourly-interval dataframe: (939913, 42)
Index(['03/01/2020 00:00:00', '308511', '3', '50', 'E', 'ML', '5', '237',
       '100', '31', '.0012', '67.5', '0', '0.1', '0.2', '0.3', '0.4', '0.5',
       '13', '.0009', '71.2', '18', '.0016', '65', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41'],
      dtype='object')
Shape of peak traffic dataframe: (1919, 22)
Index(['Dist', 'Rte', 'County', 'Postmile', 'Census Station', 'Leg', 'Year',
       'AADT', 'Diram', '1-Way PHVam', '% Kam', '% Dam', '% KDam', 'Hram',
       'Dayam', 'Dirpm', '1-Way PHVpm', '% Kpm', '% Dpm', '% KDpm', 'Hrpm',
       'Daypm'],
      dtype='object')


In [9]:
#drop data used to impute number of lanes
df.drop(df.iloc[:, 12:], axis=1, inplace=True)

#rename columns
df.columns = ['Timestamp', 'StationID', 'District', 'FreewayNum', 'DirectionOfTravel', 'LaneType', 'StationLength', 'Samples',
       '%Observed', 'TotalFlow', 'AvgOccupancy', 'AvgSpeed']
print(df.columns)
print(df.head)

Index(['Timestamp', 'StationID', 'District', 'FreewayNum', 'DirectionOfTravel',
       'LaneType', 'StationLength', 'Samples', '%Observed', 'TotalFlow',
       'AvgOccupancy', 'AvgSpeed'],
      dtype='object')
<bound method NDFrame.head of                   Timestamp  StationID  ...  AvgOccupancy  AvgSpeed
0       03/01/2020 00:00:00     308512  ...        0.0268      65.0
1       03/01/2020 00:00:00     311831  ...           NaN       NaN
2       03/01/2020 00:00:00     311832  ...           NaN       NaN
3       03/01/2020 00:00:00     311844  ...           NaN       NaN
4       03/01/2020 00:00:00     311847  ...           NaN       NaN
...                     ...        ...  ...           ...       ...
939908  03/31/2020 23:00:00    3075062  ...           NaN       NaN
939909  03/31/2020 23:00:00    3075063  ...           NaN       NaN
939910  03/31/2020 23:00:00    3075081  ...        0.0076      68.0
939911  03/31/2020 23:00:00    3075082  ...           NaN       NaN
939912  03/

In [4]:
#import data from google drive
meta = pd.read_table('/content/gdrive/My Drive/urban economics/metadata_2020_19_oct.txt')
meta.rename(columns={"ID": "StationID"}, inplace=True)
print(meta.columns)
print(meta.shape)
meta.head()

Index(['StationID', 'Fwy', 'Dir', 'District', 'County', 'City', 'State_PM',
       'Abs_PM', 'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name',
       'User_ID_1', 'User_ID_2', 'User_ID_3', 'User_ID_4'],
      dtype='object')
(1291, 18)


,StationID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,308511,50,E,3,17,NaN,31.627,60.162,38.761062,-120.569835,5.000,ML,2,Sly Park Rd,1,NaN,NaN,NaN
1,308512,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
2,311831,5,S,3,67,NaN,10.896,506.189,38.409782,-121.484120,NaN,OR,1,Elk Grove Blvd,1,NaN,NaN,NaN
3,311832,5,S,3,67,NaN,10.896,506.189,38.409782,-121.484120,NaN,FR,1,Elk Grove Blvd,1,NaN,NaN,NaN
4,311844,5,N,3,67,NaN,11.105,506.398,38.412779,-121.484363,NaN,OR,3,Elk Grove Blvd 5NB Slip,1,NaN,NaN,NaN


In [36]:
merged = df.merge(right=meta, how='inner', on='StationID')
print(merged.shape)
print(merged.columns)
merged.head(1)

(903288, 29)
Index(['Timestamp', 'StationID', 'District_x', 'FreewayNum',
       'DirectionOfTravel', 'LaneType', 'StationLength', 'Samples',
       '%Observed', 'TotalFlow', 'AvgOccupancy', 'AvgSpeed', 'Fwy', 'Dir',
       'District_y', 'County', 'City', 'State_PM', 'Abs_PM', 'Latitude',
       'Longitude', 'Length', 'Type', 'Lanes', 'Name', 'User_ID_1',
       'User_ID_2', 'User_ID_3', 'User_ID_4'],
      dtype='object')


,Timestamp,StationID,District_x,FreewayNum,DirectionOfTravel,LaneType,StationLength,Samples,%Observed,TotalFlow,AvgOccupancy,AvgSpeed,Fwy,Dir,District_y,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,03/01/2020 00:00:00,308512,3,50,W,ML,3.995,214,0,461.0,0.0268,65.0,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN


In [37]:
merged = merged.dropna(axis=0, subset=['AvgSpeed', 'TotalFlow', 'AvgOccupancy'])
merged.drop(columns=['District_y', 'District_x', 'User_ID_1', 'User_ID_2', 'User_ID_3', 'User_ID_4', 'Length'], inplace=True)

merged.isnull().sum()

Timestamp                 0
StationID                 0
FreewayNum                0
DirectionOfTravel         0
LaneType                  0
StationLength             0
Samples                   0
%Observed                 0
TotalFlow                 0
AvgOccupancy              0
AvgSpeed                  0
Fwy                       0
Dir                       0
County                    0
City                 299098
State_PM                  0
Abs_PM                    0
Latitude                  0
Longitude                 0
Type                      0
Lanes                     0
Name                      0
dtype: int64

In [38]:
print(merged.shape)
merged = merged[merged['LaneType']=='ML']
merged.drop(columns=['FreewayNum', 'DirectionOfTravel', 'LaneType'], inplace=True)
print(merged.shape)


(565553, 22)
(414914, 19)


In [39]:
merged['DateTime'] =  pd.to_datetime(merged['Timestamp'])
merged['Hour'] = merged['DateTime'].dt.hour
merged['Day'] = merged['DateTime'].dt.day
merged.Day.apply(int)
merged.Hour.apply(int)
print(merged['Hour'].describe())
print(merged['Day'].describe())
merged.to_csv('/content/gdrive/My Drive/urban economics/clean_march.csv', index=False)

count    414914.000000
mean         11.512598
std           6.918122
min           0.000000
25%           6.000000
50%          12.000000
75%          18.000000
max          23.000000
Name: Hour, dtype: float64
count    414914.000000
mean         16.143835
std           8.950248
min           1.000000
25%           8.000000
50%          16.000000
75%          24.000000
max          31.000000
Name: Day, dtype: float64


In [67]:
merged = pd.read_csv('/content/gdrive/My Drive/urban economics/clean_march.csv')
merged.columns

#merged.flowPerLane = np.nan
merged['flowPerLane'] = (merged['TotalFlow'] / merged['Lanes'])
merged['post'] = merged.Day > 12
print(merged.post[1:5])
merged = merged[merged['flowPerLane']>0]

1    False
2    False
3    False
4    False
Name: post, dtype: bool


In [92]:
y = merged['flowPerLane']
print(y.sample(5))
lnflowpl = np.log(merged['flowPerLane'])

X = merged[['County','Lanes', 'Hour', 'Fwy', 'post', 'AvgSpeed', 'AvgOccupancy']]
X[['County','Lanes', 'Hour', 'Fwy']] = X[['County','Lanes', 'Hour', 'Fwy']].astype(str)
print(X.dtypes)
dummy = pd.get_dummies(X, drop_first=True)
print(X.dtypes)


354286     361.50
375948     130.50
9350       166.50
295096     349.00
38719     1524.75
Name: flowPerLane, dtype: float64


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


County           object
Lanes            object
Hour             object
Fwy              object
post               bool
AvgSpeed        float64
AvgOccupancy    float64
dtype: object
County           object
Lanes            object
Hour             object
Fwy              object
post               bool
AvgSpeed        float64
AvgOccupancy    float64
dtype: object


In [96]:
#we rename the columns to make them easier to use
import re
newcols = []
for var in dummy.columns:
  newvar = re.sub(r'(Hour_)(.$)', r'\1 0\2', var)
  newvar = re.sub(r'(Day_)(.$)', r'\1 0\2', newvar)
  newvar = "".join(newvar.split())
  newcols.append(newvar)

print(newcols)
dummy.columns = newcols

print(dummy.columns)
dummy_csv = dummy
dummy_csv['flowpl'] = y
dummy_csv['post'] = dummy_csv['post'].astype(int)
dummy_csv['lnflowpl'] = lnflowpl
dummy_csv.to_csv('/content/gdrive/My Drive/urban economics/march_dummy.csv', index=False)

['post', 'AvgSpeed', 'AvgOccupancy', 'County_113', 'County_115', 'County_17', 'County_57', 'County_61', 'County_67', 'County_7', 'Lanes_2', 'Lanes_3', 'Lanes_4', 'Lanes_5', 'Lanes_6', 'Lanes_7', 'Hour_01', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14', 'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_02', 'Hour_20', 'Hour_21', 'Hour_22', 'Hour_23', 'Hour_03', 'Hour_04', 'Hour_05', 'Hour_06', 'Hour_07', 'Hour_08', 'Hour_09', 'Fwy_267', 'Fwy_28', 'Fwy_5', 'Fwy_50', 'Fwy_51', 'Fwy_65', 'Fwy_70', 'Fwy_80', 'Fwy_89', 'Fwy_99', 'flowpl', 'lnflowpl']
Index(['post', 'AvgSpeed', 'AvgOccupancy', 'County_113', 'County_115',
       'County_17', 'County_57', 'County_61', 'County_67', 'County_7',
       'Lanes_2', 'Lanes_3', 'Lanes_4', 'Lanes_5', 'Lanes_6', 'Lanes_7',
       'Hour_01', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14',
       'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_02',
       'Hour_20', 'Hour_21', 'Hour_22', 'Hour_23', 'Hour_03', 'Hour_0

In [97]:
ols = linear_model.LinearRegression()
ols.fit(dummy, y)
print(ols.coef_[0])
print('Coefficient of determination R^2 of the prediction: {:.3f}'.format(ols.score(dummy,y)))

lols = linear_model.LinearRegression()
lols.fit(dummy, lnfpl)
print(lols.coef_[0])
print('Coefficient of determination R^2 of the prediction: {:.3f}'.format(lols.score(dummy,y)))


3.3970625004521825e-15
Coefficient of determination R^2 of the prediction: 1.000
1.9246798087586535e-16
Coefficient of determination R^2 of the prediction: -1.658


In [90]:
def custom_sort(t):
  return t[0]

def coef_list(model, data):
  rsf_coef = []
  for i in range(len(model.coef_)):
    rsf_coef.append([list(data.columns)[i], round(model.coef_[i], 3)])
  rsf_coef.sort(key=custom_sort)
  return rsf_coef

lols_coef = coef_list(lols, dummy)
from tabulate import tabulate
print(tabulate(lols_coef, ["variable", "coefficient"], tablefmt="grid"))

coefs = pd.DataFrame(lols_coef, columns=['variable', 'coefficient'])

+--------------+---------------+
| variable     |   coefficient |
+==============+===============+
| AvgOccupancy |        13.04  |
+--------------+---------------+
| AvgSpeed     |         0.037 |
+--------------+---------------+
| County_113   |         0.497 |
+--------------+---------------+
| County_115   |         0.121 |
+--------------+---------------+
| County_17    |         0.125 |
+--------------+---------------+
| County_57    |         0.083 |
+--------------+---------------+
| County_61    |         0.371 |
+--------------+---------------+
| County_67    |         0.46  |
+--------------+---------------+
| County_7     |        -0.119 |
+--------------+---------------+
| Fwy_267      |        -0.892 |
+--------------+---------------+
| Fwy_28       |         1.132 |
+--------------+---------------+
| Fwy_5        |         0.596 |
+--------------+---------------+
| Fwy_50       |         0.476 |
+--------------+---------------+
| Fwy_51       |         0.615 |
+---------

In [84]:
coefs.to_csv('/content/gdrive/My Drive/urban economics/log_flow_ols_coefs.csv', index=False)

In [95]:
dummy.post.describe()

count     414712
unique         2
top         True
freq      257173
Name: post, dtype: object